In [1]:
from obspy.core import Stream, read
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
from matplotlib.colors import Normalize
import pandas as pd
import os 
import os.path

In [2]:
path_to_files = '/Users/sydneydybing/StrainProject/2024/'

earthquakes = pd.read_csv(path_to_files + 'M6_catalog_2004-2024.csv', dtype = str)

weird_events_only = False
if weird_events_only:
    earthquakes = earthquakes.iloc[[4,15]]

small_test = False
if small_test:
    earthquakes = earthquakes[5:7]
    
redo_picks = True
if redo_picks:
    earthquakes = earthquakes.iloc[[3,5,6,8,10,17,19,20,21,24,25,26,27,29]]

origin_times = earthquakes.time.values
mags = earthquakes.mag.values

bsm_mda = pd.read_csv(path_to_files + 'bsm_metadata.csv')
NA_only = bsm_mda.loc[bsm_mda['LONG'] < -110]
stas = NA_only['BNUM'].values

eqs_test = earthquakes[4:5]
ots_test = origin_times[4:5]
mags_test = mags[4:5]
rows_for_extra_times = [8,9,14,16]

dist_array = np.load('/Users/sydneydybing/StrainProject/2024/NA_evt-sta_dist_array_less500km_withdata.npy')

# Array order:

# 0. Earthquake identifier
# 1. Earthquake origin time
# 2. Earthquake latitude
# 3. Earthquake longitude
# 4. Earthquake magnitude
# 5. Station name
# 6. Station latitude
# 7. Station longitude
# 8. Hypocentral distance (km)

In [3]:
dist_array.shape

(439, 9)

In [3]:
cmap = plt.get_cmap('turbo', 500) 

for idx in range(len(earthquakes)):
    
    no_data = False
    
    if idx >= 31: # No data available for 4 earliest earthquakes
        no_data = True
        
    if no_data == False:
        fig = plt.figure(figsize = (10,6), dpi = 300, facecolor = 'white', num = 1, clear = True)
        ax = fig.add_subplot(111)
        
    ot = origin_times[idx]
    print('Earthquake ' + str(idx+1) + '/' + str(len(mags)))
    
    if idx in rows_for_extra_times:
        cut_ot = ot[:13]
    else:
        cut_ot = ot[:10]
    
    eq_id = str(cut_ot) + '_M' + str(mags[idx])
#     print(eq_id)
        
    # Find rows that match this event in the array with the hypocentral distances
    
    i = np.where(dist_array[:,0] == eq_id)[0]
#     print(dist_array[i])
        
    for sta in stas:
        
        try:
            
            # Peak strain data
        
            pst = read(path_to_files + 'strain_data/peak/' + str(cut_ot) + '_M' + str(mags[idx]) + '/' + str(sta) + '.mseed')
                
        except:
            continue # skips to next station

        # Find row that matches this station within the rows of the right event

        j = np.where(dist_array[i,5] == sta)[0]

#             print(dist_array[i][j])

        hypdist = float(dist_array[i][j][0][8])
#         print(hypdist)
        
        int_hypdist = int(hypdist)
#         print(int_hypdist)

        pst_times = pst[0].times()
        pst_data = pst[0].data

        # Plot stations, but avoid plotting the weird stations for the two problem earthquakes

        if idx == 4 and sta == 'B072' or sta == 'B076':
            print('Skipping ' + str(cut_ot) + ' ' + str(sta))

        elif idx == 15 and sta == 'B928':
            print('Skipping ' + str(cut_ot) + ' ' + str(sta))

        else:
            ax.set_title('Earthquake ' + str(cut_ot) + '_M' + str(mags[idx]), fontsize = 18)
            ax.plot(pst_times, pst_data*10**9, color = cmap(int_hypdist-1), label = sta)
            ax.set_yscale('log')
            ax.set_ylabel('Log of peak extensional\nmicrostrain ($10^{-6}$)', fontsize = 15)
            ax.set_xlabel('Time (s)', fontsize = 15)
            ax.set_xlim(0,120)
            ax.tick_params(labelsize = 13)
            
    # Plot after each earthquake
    if no_data: 
        pass
    else:
        cbar = fig.colorbar(plt.cm.ScalarMappable(norm = Normalize(0, 500), cmap = cmap), ax = ax)
        cbar.ax.tick_params(labelsize = 11)
        cbar.ax.set_ylabel(ylabel = 'Hypocentral distance (km)', fontsize = 13)
        ax.legend(loc = 'lower right', ncol = 2, fontsize = 12)
#         plt.show();
        plt.savefig(path_to_files + 'figures/peak_strain_plots/NA_M6_500km/rainbow_by_hypdist/' + str(cut_ot) + '_M' + str(mags[idx]) + '.png', format = 'PNG')
        plt.close();


Earthquake 1/14
Earthquake 2/14
Earthquake 3/14


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Earthquake 4/14
Earthquake 5/14
Earthquake 6/14
Earthquake 7/14
Skipping 2014-08-24 B076
Earthquake 8/14


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Earthquake 9/14


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Earthquake 10/14
Earthquake 11/14
Earthquake 12/14
Earthquake 13/14
Earthquake 14/14
